First, Lets download some packages:

In [13]:
# check to scan

import subprocess

# This is our shell command, executed by Popen.

p = subprocess.Popen("Start-MpScan -ScanType FullScan", stdout=subprocess.PIPE, shell=True)

print(p.communicate())

(b'', None)


In [1]:
!pip install prettytable
!pip install wmi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now, Lets import them and some more libraries:

In [2]:
import psutil
import time
from subprocess import call
from prettytable import PrettyTable
#import wmi

Now, Lets gather some data and display it in tables:

# ***Battery:*** 

In [4]:
#c = wmi.WMI()
#t = wmi.WMI(moniker = "//./root/wmi")

# Run an infinite loop to constantly monitor the system
def print_battery_stat():
    print("==============================Process Monitor\
        ======================================")

    # Fetch the battery information
    battery = psutil.sensors_battery().percent
    print("Battery Available: %d " % (battery,) + "%")

    batts1 = c.CIM_Battery(Caption='Portable Battery')
    for i, b in enumerate(batts1):
        print('Battery %d Design Capacity: %d mWh' % (i, b.DesignCapacity or 0))

    batts = t.ExecQuery('Select * from BatteryFullChargedCapacity')
    for i, b in enumerate(batts):
        print('Battery %d Fully Charged Capacity: %d mWh' %
              (i, b.FullChargedCapacity))

    batts = t.ExecQuery('Select * from BatteryStatus where Voltage > 0')
    for i, b in enumerate(batts):
        print('Voltage:           ' + str(b.Voltage))
        print('RemainingCapacity: ' + str(b.RemainingCapacity))

        if b.Charging:
            raise Exception("Unplug charging cable during measurements!")


# ***Network:*** 

In [5]:
def print_network_stat():
    print("----Networks----")
    table = PrettyTable(['Network', 'Status', 'Speed'])
    for key in psutil.net_if_stats().keys():
        name = key
        up = "Up" if psutil.net_if_stats()[key].isup else "Down"
        speed = psutil.net_if_stats()[key].speed
        table.add_row([name, up, speed])
    print(table)

# ***Memory:*** 

In [6]:
def create_total_memory_table():
    print("----Memory----")
    memory_table = PrettyTable(["Total(GB)", "Used(GB)",
                                "Available(GB)", "Percentage"])
    vm = psutil.virtual_memory()
    memory_table.add_row([
        f'{vm.total / 1e9:.3f}',
        f'{vm.used / 1e9:.3f}',
        f'{vm.available / 1e9:.3f}',
        vm.percent
    ])
    print(memory_table)

# ***Disk:*** 

In [7]:
def create_total_disk_table():
    print("----Disk----")
    disk_table = PrettyTable(["Total(GB)", "Used(GB)",
                              "Available(GB)", "Percentage"])
    disk_stat = psutil.disk_usage('/')
    disk_table.add_row([
        f'{disk_stat.total / 1e9:.3f}',
        f'{disk_stat.used / 1e9:.3f}',
        f'{disk_stat.free / 1e9:.3f}',
        disk_stat.percent
    ])
    print(disk_table)

# ***Disk I/O:*** 

In [8]:
# TODO: i/o data is cumulative. maybe we should subtract that from initial i/o data (when the python program starts)
def create_current_disk_io_table():
    print("----Disk I/O----")
    disk_table = PrettyTable(["read_count", "write_count",
                              "read_bytes(GB)", "write_bytes(GB)"])
    disk_io_stat = psutil.disk_io_counters()
    disk_table.add_row([
        disk_io_stat.read_count,
        disk_io_stat.write_count,
        f'{disk_io_stat.read_bytes / 1e9:.3f}',
        f'{disk_io_stat.write_bytes / 1e9:.3f}'
    ])
    print(disk_table)


# ***All Processes:*** 

In [9]:
# TODO: i/o data is cumulative. maybe we should subtract that from initial i/o data (when the python program starts)
def create_process_table():
    print("----Processes----")
    process_table = PrettyTable(['PID', 'PNAME',
                                 'CPU', 'NUM THREADS', 'MEMORY(MB)', 'MEMORY(%)', 'read_count', 'write_count', 'read_bytes', 'write_bytes'])

    proc = []
    # get the pids from last which mostly are user processes
    for p in psutil.process_iter():
        try:
            # trigger cpu_percent() the first time which leads to return of 0.0
            p.cpu_percent()
            proc.append(p)

        except Exception as e:
            pass

    # sort by cpu_percent
    top = {}
    time.sleep(0.1)
    for p in proc:
        # trigger cpu_percent() the second time for measurement
        top[p] = p.cpu_percent() / psutil.cpu_count()

    top_list = sorted(top.items(), key=lambda x: x[1])
    top10 = top_list[-20:]
    top10.reverse()

    for p, cpu_percent in top10:

        # While fetching the processes, some of the subprocesses may exit
        # Hence we need to put this code in try-except block
        try:
            # oneshot to improve info retrieve efficiency
            with p.oneshot():
                disk_io = p.io_counters()
                process_table.add_row([
                    str(p.pid),
                    p.name(),
                    f'{cpu_percent:.2f}' + "%",
                    p.num_threads(),
                    f'{p.memory_info().rss / 1e6:.3f}', # TODO: maybe should use uss instead rss?
                    round(p.memory_percent(),2),
                    disk_io.read_count,
                    disk_io.write_count,
                    disk_io.read_bytes,
                    disk_io.write_bytes
                ])

        except Exception as e:
            pass
    print(process_table)


# ***CPU:*** 

In [ ]:
# TO DO: calculate the total cpu consumption

Now, Lets run the main:

In [3]:
#print_battery_stat()
create_total_disk_table()

for i in range(5):

    # Clear the screen using a bash command
    #call('clear')


    # Fetch the Network information - relevant?
    # print_network_stat()

    create_current_disk_io_table()

    # Fetch the memory information
    create_total_memory_table()


    # Fetch the 10 processes from available processes that has the highest cpu usage
    create_process_table()

    # Create a 1 second delay
    time.sleep(1)

#print_battery_stat()


----Disk----
+-----------+----------+---------------+------------+
| Total(GB) | Used(GB) | Available(GB) | Percentage |
+-----------+----------+---------------+------------+
|  115.658  |  41.211  |     74.431    |    35.6    |
+-----------+----------+---------------+------------+
----Disk I/O----


ValueError: ignored